# 문서 군집화(Clustering)와 문서간 유사도(Similarity) 측정

* 문서 군집화 : 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류

* ref : https://techblog-history-younghunjo1.tistory.com/114

* BOW .Count Vectorizer : 단어 발생 빈도수에만 초점, 문서들간의 관계 파악 X
* Tf-idf 사용
* 벡터화 방법
    - BOW(Bag Of Words) : 단어 발생 빈도수에 기반, Feature 벡터화
    - Word embedding : 단어들간의 의미 관계(벡터들간의 방향) 고려해 벡터화

In [24]:
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
import seaborn as sns

#-------------------- 주피터 셀 넓이 조절 ---------------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', None)

sns.set()

In [ ]:
# from sklearn.datasets import fetch_20newsgroups
# news_data = fetch_20newsgroups(subset='all',random_state=156)

In [38]:
path = './datasets/UCI_Opinion_Review'
all_files = glob.glob(os.path.join(path,"*.data"))
# all_files = os.listdir('./datasets/UCI_Opinion_Review')

filename_list = []
opinion_text = []
for file_ in all_files:
   #['', '/datasets/UCI_Opinion_Review\\accuracy_garmin_nuvi_255W_gps', 'txt', 'data']
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    filename_list.append(filename)
    
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    opinion_text.append(df.to_string())

document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head(1)

filename  \
0  accuracy_garmin_nuvi_255W_gps   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [48]:
# nltk.download('punkt')     # c:\ai\pythonproject\venv\nltk_data
# nltk.download('wordnet')
  

[nltk_data] Downloading package wordnet to
[nltk_data]     c:\ai\pythonproject\venv\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [49]:
# 텍스트 단어들의 어근 원형을 추출하기 위해 함수 생성
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string

# ord('문자열') => 문자열의 ASCII코드를 반환
# dict(key, value)형태로 모든 구두점의 각 ASCII코드를 key값으로 넣어주자!
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]
def LemNormalize(text):
    #구두점 삭제해준 상태로 토큰화
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                             stop_words='english', 
                             ngram_range=(1,2),
                             min_df=0.05, max_df=0.85)              # 빈도수 0.05이하, 0.85이상의 빈도수 단어들 제거
ftr_vect = tfidf_vect.fit_transform(document_df['opinion_text'])    # 벡터화

c:\ai\pythonproject\venv\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [54]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, max_iter=10000, random_state=42)
cluster_label = kmeans.fit_predict(ftr_vect)
document_df['cluster_label'] = cluster_label
document_df.sort_values(by=['cluster_label']).head(2)

filename  \
25  performance_honda_accord_2008   
29      quality_toyota_camry_2007   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [55]:
# 문서의 feature(단어별) cluster_centers_
cluster_centers = kmeans.cluster_centers_
print(cluster_centers.shape)  #(클러스터 레이블, 벡터화feature(단어들)
print(cluster_centers)

(3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


In [56]:
def get_cluster_details(cluster_model, cluster_data, feature_names,
                       cluster_num, top_n_features=10):
    cluster_details = {}
    center_feature_idx = cluster_model.cluster_centers_.argsort()[:,::-1]  #내림차순정렬
    
    for cluster_num in range(cluster_num):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        top_ftr_idx = center_feature_idx[cluster_num, :top_n_features]   # 인덱스 top10
        top_ftr = [feature_names[idx] for idx in top_ftr_idx]            # 상위 10개 feature들의 center값들 반환
        top_ftr_val = cluster_model.cluster_centers_[cluster_num, top_ftr_idx].tolist()
        
        cluster_details[cluster_num]['top_features'] = top_ftr           # 개별 군집 정보 넣기
        cluster_details[cluster_num]['top_featrues_value'] = top_ftr_val
        filenames = cluster_data[cluster_data['cluster_label']==cluster_num]['filename']  ## 해당 cluster_num으로 분류된 파일명 넣기
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames    
    return cluster_details

def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print(f"#####Cluster Num: {cluster_num}")
        print()
        print("상위 10개 feature단어들:\n", cluster_detail['top_features'])
        print()
        print(f"Cluster {cluster_num}으로 분류된 문서들:\n{cluster_detail['filenames'][:5]}")
        print('-'*20)

feature_names = tfidf_vect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=kmeans,
                                     cluster_data=document_df,
                                     feature_names=feature_names,
                                     cluster_num=3,
                                     top_n_features=10)
print_cluster_details(cluster_details)

#####Cluster Num: 0

상위 10개 feature단어들:
 ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']

Cluster 0으로 분류된 문서들:
['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007']
--------------------
#####Cluster Num: 1

상위 10개 feature단어들:
 ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']

Cluster 1으로 분류된 문서들:
['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo']
--------------------
#####Cluster Num: 2

상위 10개 feature단어들:
 ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']

Cluster 2으로 분류된 문서들:
['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_